In [24]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys

import numpy as np

# Root directory of the project
from PIL import Image

ROOT_DIR = os.path.abspath(".")

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
import skimage.io

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

import samples.coco.coco as coco
import samples.police2.backlash as backlash


class MaskRCNNModel():
    MODEL_FILE_NAME = "mask_rcnn_coco.h5"
    CLASSES = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

    CONFIG_CLASS = coco.CocoConfig

    def __init__(self):
        # Local path to trained weights file
        COCO_MODEL_PATH = os.path.join(ROOT_DIR, self.MODEL_FILE_NAME)

        # Download COCO trained weights from Releases if needed
        if not os.path.exists(COCO_MODEL_PATH):
            utils.download_trained_weights(COCO_MODEL_PATH)

        class InferenceConfig(self.CONFIG_CLASS):
            # Set batch size to 1 since we'll be running inference on
            # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1
            NUM_CLASSES = len(self.CLASSES)  # COCO has 80 classes

        self.config = InferenceConfig()
        #         config.display()

        # Create model object in inference mode.
        model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=self.config)

        # Load weights
        model.load_weights(COCO_MODEL_PATH, by_name=True)

        # COCO Class names
        # Index of the class in the list is its ID. For example, to get ID of
        # the teddy bear class, use: class_names.index('teddy bear')
        class_names = self.CLASSES

        self.model = model

model_full = MaskRCNNModel()


In [26]:
from tqdm import tqdm
import numpy as np
from imantics import Polygons, Mask

import json
project = json.load(open("via-project-template.json"))

import glob
images_path = "/Users/skabbit/Projects/backlash/backlash/datasets/usa-police"
files = sorted(glob.glob(images_path + "/*.jpg"))

data = {}

for filename in tqdm(files):
#     print(filename)
    image = skimage.io.imread(filename)
    results = model_full.model.detect([image], verbose=0)
    
    masks = results[0]['masks'][:,:,results[0]['class_ids'] == 1] #  & results[0]['score'] > 0.9 where?
    regions = []
    
#     print("Regions:", masks.shape[2])
    for i in range(masks.shape[2]):
        mask = masks[:,:,i]
        polygons = Mask(mask).polygons()
        polygons.simplify()
        shape_attributes = {
            "name": "polygon",
            "all_points_x": [],
            "all_points_y": []
        }
        for point in polygons.points[0]:
            shape_attributes['all_points_x'].append(int(point[0]))
            shape_attributes['all_points_y'].append(int(point[1]))        
        regions.append({
            "shape_attributes": shape_attributes,
            "region_attributes": {},
        })

    fn = os.path.split(filename)[1]
    size = os.path.getsize(filename)
    data[fn + str(size)] = {
        "filename": fn,
        "size": size,
        "file_attributes": {},
        "regions": regions
    }

project['_via_img_metadata'] = data
project["_via_image_id_list"] = list(data.keys())
project['_via_settings']['core']['default_filepath'] = images_path+"/"
project['_via_settings']['core']['filepath'] = {images_path+"/": 1}

json.dump(project, open("via-project.json", "w"))

100%|██████████| 664/664 [1:06:44<00:00,  6.03s/it]
